In [1]:
import numpy as np 
import sys
sys.path.append("../")
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.multioutput import MultiOutputClassifier

In [2]:
from pyhealth.datasets import MIMIC3BaseDataset, MIMIC4BaseDataset, eICUBaseDataset, OMOPBaseDataset
base_dataset = MIMIC3BaseDataset(root="../srv/local/data/physionet.org/files/mimiciii/1.4")
# base_dataset = eICUBaseDataset(root="/srv/local/data/physionet.org/files/eicu-crd/2.0")
# base_dataset = MIMIC4BaseDataset(root="/srv/local/data/physionet.org/files/mimiciv/2.0/hosp")
# base_dataset = OMOPBaseDataset(root="/srv/local/data/zw12/pyhealth/raw_data/synpuf1k_omop_cdm_5.2.2")
base_dataset.info()

/home/pj20/miniconda3/envs/pyhealth/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



        ----- Output Data Structure -----
        Dataset.patients: [
            {
                patient_id: patient_id, 
                visits: [
                    {
                        visit_id: visit_id, 
                        patient_id: patient_id, 
                        conditions: [List], 
                        procedures: [List],
                        drugs: [List],
                        visit_info: <dict>
                    }
                    ...
                ]                    
            } 
            ...
        ]
        


In [3]:
from pyhealth.tasks import DrugRecDataset
drug_rec_dataset = DrugRecDataset(base_dataset)
drug_rec_dataset.info()


        ----- Output Data Structure -----
        >> drug_rec_dataloader[0]
        >> {
            "conditions": List[tensor],
            "procedures": List[tensor],
            "drugs": List[tensor]
        }
        


In [4]:
from pyhealth.models import MLModel

lr_model = MLModel(drug_rec_dataset, 'LR')

In [5]:
# Train
lr_model.train(0.1)

Starting fold:  0
Mean loss across folds:  0.11795481888110033
STD  loss across folds:  0.0
Starting fold:  1
Mean loss across folds:  0.12007267085260966
STD  loss across folds:  0.0021178519715093214
Starting fold:  2
Mean loss across folds:  0.12064960767932104
STD  loss across folds:  0.001912043458235771
Starting fold:  3
Mean loss across folds:  0.12141790203723198
STD  loss across folds:  0.0021243260815679886
Starting fold:  4
Mean loss across folds:  0.12111781850016043
STD  loss across folds:  0.0019925886573875363


In [9]:
gt = lr_model.y_test[:100]
pred_res = lr_model.predict(lr_model.X_test[:100])

In [12]:
count = 0
res = pred_res
ground_truth = gt
zeros = np.zeros(len(pred_res), dtype=int)

for i in range(len(res)):
    if (np.sum(np.absolute(res[i] - ground_truth[i]))) < 100:
        count += 1
#         print(res[i])
print(count)

73


In [13]:
svm = MLModel(drug_rec_dataset, 'SVM')

In [14]:
svm.train(0.1)

Starting fold:  0
Mean loss across folds:  0.06941270445217748
STD  loss across folds:  0.0
Starting fold:  1
Mean loss across folds:  0.07095738444084362
STD  loss across folds:  0.0015446799886661422
Starting fold:  2
Mean loss across folds:  0.07103016799812382
STD  loss across folds:  0.0012654191943164432
Starting fold:  3
Mean loss across folds:  0.07144105797639207
STD  loss across folds:  0.0013066966081429132
Starting fold:  4
Mean loss across folds:  0.07128432612347527
STD  loss across folds:  0.0012100513689346757


In [15]:
gt = svm.y_test[:100]
pred_res = svm.predict(svm.X_test[:100])

In [16]:
count = 0
res = pred_res
ground_truth = gt
zeros = np.zeros(len(pred_res), dtype=int)

for i in range(len(res)):
    if (np.sum(np.absolute(res[i] - ground_truth[i]))) < 100:
        count += 1
#         print(res[i])
print(count)

68


In [17]:
count = 0
for i in range(len(res)):
    if (np.sum(np.absolute(res[i] - ground_truth[i]))) < 30:
        count += 1
#         print(res[i])
print(count)

2
